# Чтение и запись данных. Часть 2

## 1. Работа с базой данных из pandas


### 1.1 

Подготовка данных:

In [1]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

# получаем имена таблиц из базы
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

df = pd.read_sql_query("select * from public.ratings limit 5;", conn)
df.head()

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('exploratory',), ('course_purchases',)]


,userid,movieid,rating,timestamp
0,1003,724,3.5,1236374651
1,1003,736,2.0,1236376434
2,1003,780,3.0,1236377019
3,1003,832,3.0,1236376544
4,1003,912,2.5,1236375873


**Задание простого уровня** <br>Посчитайте количество записей в каждой из таблиц **keywords, links, ratings** и выведите в формате имя *таблицы:количество строк*

*Результат работы:*
<pre>
table keywords:92838
table links:91686
table ratings:1555552
</pre>

In [2]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

table_names = [i[0] for i in tables_data]
for table in table_names[:3]:
        
        sql_str = "SELECT COUNT(*) FROM public.{} LIMIT 1;".format(table)
        cursor.execute(sql_str)
        row_count = [a for a in cursor.fetchall()][0][0]
        print("table {}:{}".format(table, row_count))

cursor.close()

table keywords:92838
table links:91686
table ratings:1555552


### 1.2

**Задание среднего уровня** <br>Напечатайте колонки таблицы *ratings*. Подсказка: поисследуйте объект **cursor** с помощью встроенной в python команды *dir()*

In [3]:
import pandas as pd
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

#1 вариант. Мне кажется проще 
print("------------1 вариант--------------")
df = pd.read_sql_query("select * from public.ratings limit 5;", conn)

table_rating_fields = []
for col in list(df.columns):
    table_rating_fields.append(col)
print("Поля таблицы ratings: %s" % table_rating_fields)

print('------------------------------------')
print("------------2 вариант--------------")

sql_str = "SELECT * FROM ratings LIMIT 10;"
cursor.execute(sql_str)
conn.commit()

table_rating_fields_2 = []
for i in range(len(cursor.description)):
    desc = cursor.description[i]
    table_rating_fields_2.append(desc[0])
print("Поля таблицы ratings: %s" % table_rating_fields)

cursor.close()

------------1 вариант--------------
Поля таблицы ratings: ['userid', 'movieid', 'rating', 'timestamp']
------------------------------------
------------2 вариант--------------
Поля таблицы ratings: ['userid', 'movieid', 'rating', 'timestamp']


### 1.3

**Задание высокого уровня**
* Посмотрите на столбцы таблицы **links** и столбцы таблицы **ratings** по какому полю можно заджойнить эти две таблицы
* Посчитайте количество фильмов из links, у который нет оценок в ratings.

In [4]:
import pandas as pd
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()


#джойн можно провести по колонке movieid. Однако как этим воспользоваться, я не придумал.
#sql_str = "SELECT DISTINCT(links.movieid) from links JOIN ratings ON links.movieid = ratings.movieid order by movieid;"

# Поэтому просто посчитал общее кол-во id фильмов, количество id c оценками и посчитал разницу. 

sql_str_1 = 'select DISTINCT(movieid) from links order by movieid'
sql_str_2 = 'select DISTINCT(movieid) from ratings order by movieid'

df_1 = pd.read_sql_query(sql_str_1, conn)
df_2 = pd.read_sql_query(sql_str_2, conn)
all_id = df_1.size
id_with_rating = df_2.size

print("Количество фильмов без рейтингов: %s" %(all_id - id_with_rating))

Количество фильмов без рейтингов: 28333


## 2. Работа с БД: MongoDB

### 2.1

Подготовка данных

In [5]:
from pymongo import MongoClient

mongo_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 27017,
    "user": "students",
    "password": "X63673t47Gl03Sq",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Коллекции, доступные в MongoDB: ['tags', 'users']
Число документов в коллекции 158680


**Задание простого уровня** <br>Посчитайте количество контента, у которого встречается тэг "toy"

* сделайте выборку такого контента (а точнее - поля `id`) в питоновский `list`
* подсчитайте количество элементов в `list`

In [6]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}

mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'})

cursor_items = [item for item in mongo_cursor]


print("Количество контента с тэгом 'toy': %s\n" % len(cursor_items))


        

Количество контента с тэгом 'toy': 19



### 2.2

**Задание среднего уровня** <br> Выведите **только названия** 10 самых непопулярных тэгов <br>

Подсказка: посмотрите в уроке, как MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

*Результат работы*
<pre>
Счётчик тэгов ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']
</pre>

In [7]:
pipline = [
    {"$group":
        {"_id": "$name"}
     },
    {"$sort":
        {"tag_count": 1}
     },
    {"$limit": 10}
]


print('Топ 10 непопулярных тегов: %s' %([i['_id'] for i in collection.aggregate(pipline)]))

Топ 10 непопулярных тегов: ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']


### 2.3

**Задание высокого уровня**

1. Подключитесь к Postgres. Найдите top-10 самого популярного контента
1. Сохраните в файл `content_popularity.json` в виде
<pre>
{'movieId': 931, 'popularity': 999}
{'movieId': 429, 'popularity': 111}
</pre>

Этот формат называется single-line json и его можно загрузить в MongoDB с помощью команды
<pre>
mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file content_popularity.json
</pre>

**ВНИМАНИЕ!** Это просто пример как можно использовать single-line JSON, заливать его никуда не нужно - в задаче требуется только подготовить файл в правильном виде

*Подсказка* воспользуйтесь функцией `json.dumps`

*Результат работы* - файл `content_popularity.json`:

<pre>
{"movieId": 318, "popularity": 5626}
{"movieId": 356, "popularity": 5464}
{"movieId": 296, "popularity": 5242}
{"movieId": 593, "popularity": 5048}
{"movieId": 2571, "popularity": 4718}
{"movieId": 260, "popularity": 4556}
{"movieId": 480, "popularity": 4540}
{"movieId": 527, "popularity": 4120}
{"movieId": 1, "popularity": 4072}
{"movieId": 110, "popularity": 3928}
</pre>

In [2]:
import pandas as pd
import psycopg2
import json

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "select movieid, count(rating) as popularity from public.ratings group by movieid order by movieid;"
df = pd.read_sql_query(sql_str, conn)

cursor.close()
# для скорости ограничил таблицу 10 значениями
a = df.head(10)
content_popularity = [{"movieId":int(i), "popularity":int(j)} for (i,j) in a.values]

user_json = json.dumps(content_popularity)

print("Объект {} : {}".format(type(user_json), user_json))

with open("./content_popularity.json", "w") as file:
    file.write(user_json)

Объект <class 'str'> : [{"movieId": 1, "popularity": 4072}, {"movieId": 2, "popularity": 1584}, {"movieId": 3, "popularity": 878}, {"movieId": 4, "popularity": 172}, {"movieId": 5, "popularity": 920}, {"movieId": 6, "popularity": 1642}, {"movieId": 7, "popularity": 952}, {"movieId": 8, "popularity": 100}, {"movieId": 9, "popularity": 284}, {"movieId": 10, "popularity": 1972}]
